### Libraries

In [1]:
!pip install -qU langchain-community faiss-cpu
!pip install -qU langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [2]:
import pandas as pd

### CSV file transfroming to Pandas dataframe

In [3]:
df = pd.read_csv("../data/raw/goods.csv", index_col=0)
df_unique = df.drop_duplicates("name")

Dataframe structure

In [4]:
df_unique.sample(4)

,name,category,price,rating,url,description
10038,"Лента репсовая, 15 мм, 23 ± 1 м, цвет кораллов...",Ленты,39.0,5.0,https://www.sima-land.ru/9894987/lenta-repsova...,NaN
6974,"Холст на подрамнике, хлопок 100%, 1,6 х 50 х 7...",Товары для художников,862.0,4.6,https://www.sima-land.ru/7098440/holst-na-podr...,Холст на подрамнике используется для работы ма...
12916,"Наклейка пластик стразы ""Капельки разных разме...",Принадлежности для творчества,75.0,0.0,https://www.sima-land.ru/9735532/nakleyka-plas...,NaN
106,Картина по номерам на холсте с подрамником «Лю...,Картины по номерам,890.0,5.0,https://www.sima-land.ru/9846472/kartina-po-no...,Написанная своими руками картина украсит интер...


In [5]:
print(df_unique.shape[0])                                       # number of items
print(df_unique[df_unique['description'].isna()].shape[0])      # number of items without description

14615
5447


In [6]:
df_unique.to_csv("../data/processed/goods_unique.csv", sep='\t')

### Formatting Pandas dataframe to Langchain Documents

In [7]:
from langchain_core.documents import Document


docs = []

for row in df_unique[~df_unique['description'].isna()].itertuples():
    doc = Document(
        page_content=f"Название: {row.name}. Описание: {row.description}",
        metadata={
            "url": row.url,
            "category": row.category,
            "rating": row.rating,
            "price": row.price,
            "name": row.name
        },
    )
    docs.append(doc)

for row in df_unique[df_unique['description'].isna()].itertuples():
    doc = Document(
        page_content=f"Название: {row.name}. Описание: описание отсутствует",
        metadata={
            "url": row.url,
            "category": row.category,
            "rating": row.rating,
            "price": row.price,
            "name": row.name
        },
    )
    docs.append(doc)

FAISS index creation

In [8]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings


embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

db = FAISS.from_documents(docs, embeddings)
db.save_local("../data/processed/faiss_index")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Some examples

In [9]:
results = db.similarity_search("Люблю рисовать", 5)
for res in results:
    print(res.metadata['name'])

Раскраска коврик «Любимые сказки», рисуем водой, с маркером
Книга «Я учусь рисовать. Для девочек», 16 стр.
Книжка для рисования водой «Рисуем транспорт», с маркером
Раскраска «Песнь фантазии», 16 стр., Аниме
Раскраска детская «Рисуем по номерам», 12 стр.


In [10]:
results = db.similarity_search("Люблю шить", 5)
for res in results:
    print(res.metadata['name'])

Машина для вязания «Зимушка»
Набор для шитья. Игрушка из фетра своими руками «‎Любимый утенок»‎
Прищепки для шитья «С сердцем», 3,5 × 2 × 1 см, 10 шт, цвет красный
Набор для шитья «Шью-шью», пухляш Шайни
Набор для шитья «Шью-шью», Зайка


In [11]:
results = db.similarity_search("Люблю музыкальные инструменты", 5)
for res in results:
    print(res.metadata['name'])

Чип музыкальный «Любимая игрушка», круглый, звук
Детская музыкальная игрушка «Свистулька керамическая. Соловушка», МИКС
Детская музыкальная игрушка «Свистулька. Птичка»
Сувенирный музыкальный инструмент Бубен «Уникальность», 23×23×7,5 см
Инструмент для квиллинга МИКС


In [12]:
results = db.similarity_search(
    """
      Я хочу красиво украсить подарок для близкого человека
      Я не хотел бы видеть рисунки и сердечки, но мне нравятся ленты
      Близкий человек любит классический стиль и розовый цвет
      Ленты не должны быть слишком широкие, лучше атласные
    """,
    5
)
for res in results:
    print(res.metadata['name'])

Лента атласная, 6 мм, 23±1 м, цвет розовое золото
Лента атласная, 40 мм, 23±1 м, цвет розовое золото
Лента атласная, 40 мм × 23 ± 1 м, цвет розовый антик №102
Лента атласная, 50 мм, 23±1 м, цвет розовое золото
Лента атласная, 40 мм × 23±1 м, светло-розовая
